In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import numpy as np

In [2]:
#change bike_lanes file path to citibike-publicspace
bike_lanes = gpd.read_file('../data/external/bike-lanes/nyc_bike_routes_2016.shp')
stations = pd.read_csv('../data/processed/stations.csv')

In [3]:
bike_lanes.columns

Index(['OBJECTID', 'SegmentID', 'allclasses', 'bikedir', 'boro', 'comments',
       'fromstreet', 'ft_facilit', 'geometry', 'instdate', 'lanecount',
       'moddate', 'onoffst', 'street', 'tf_facilit', 'tostreet'],
      dtype='object')

In [4]:
bike_lanes = bike_lanes.drop(['instdate', 'moddate'], 1)

In [5]:
bike_lanes = bike_lanes.to_crs({'init': 'epsg:4326'})

In [6]:
geometry = gpd.GeoSeries([Point(xy) for xy in zip(stations.Longitude, stations.Latitude)])
geometry = geometry.buffer(.0005)
geo_stations = gpd.GeoDataFrame(stations, geometry=geometry)
geo_stations.crs = {'init' :'epsg:4326'}
geo_stations.head()

,Station_id,Station_Name,Location,Latitude,Longitude,geometry
0,72,W 52 St & 11 Ave,W 52 St & 11 Ave,40.767272,-73.993929,"POLYGON ((-73.99342888 40.76727216, -73.993431..."
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74...."
2,82,St James Pl & Pearl St,St James Pl & Pearl St,40.711174,-74.000165,"POLYGON ((-73.99966544999999 40.71117416, -73...."
3,83,Atlantic Ave & Fort Greene Pl,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,"POLYGON ((-73.97582328 40.68382604, -73.975825..."
4,116,W 17 St & 8 Ave,W 17 St & 8 Ave,40.741776,-74.001497,"POLYGON ((-74.00099745999999 40.74177603, -74...."


In [7]:
stations_bike_quality = gpd.sjoin(geo_stations, bike_lanes, op='intersects')
stations_bike_quality.head()

,Station_id,Station_Name,Location,Latitude,Longitude,geometry,index_right,OBJECTID,SegmentID,allclasses,bikedir,boro,comments,fromstreet,ft_facilit,lanecount,onoffst,street,tf_facilit,tostreet
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9448,9490,0031767,II,L,1.0,None,BEACH ST,None,1.0,ON,W BROADWAY,Standard,READE ST
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9451,9493,0031796,II,L,1.0,None,BEACH ST,None,1.0,ON,W BROADWAY,Standard,READE ST
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9488,9530,0110911,II,L,1.0,None,BEACH ST,None,1.0,ON,VARICK ST,Standard,W BROADWAY
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9450,9492,0031798,II,L,1.0,None,BEACH ST,None,1.0,ON,W BROADWAY,Standard,READE ST
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9487,9529,0031789,II,L,1.0,None,BEACH ST,None,1.0,ON,VARICK ST,Standard,W BROADWAY


In [8]:
#bike direction
ft_facilit_keys = stations_bike_quality['ft_facilit'].unique()
tf_facilit_keys = stations_bike_quality['tf_facilit'].unique()
print(ft_facilit_keys)
print(tf_facilit_keys)

[None 'Protected Path' 'Bike-Friendly Parking' 'Curbside' 'Standard'
 'Sharrows' 'Greenway' 'Sidewalk' 'Ped Plaza' 'Signed Route'
 'Opposite Sidewalk']
['Standard' None 'Bike-Friendly Parking' 'Curbside' 'Sharrows'
 'Protected Path' 'Ped Plaza' 'Greenway' 'Sidewalk'
 'Sharrows/Protected Path' 'Signed Route' '<Null>' 'Sharrows/Standard']


ft_facility = feature facility start point to end point

bikedirR = ride start point to end point

tf_facility = feature facility end point to start point

bikedirL = ride end point to start point

bikedir2 = two-way

protected bike lanes: http://www.nyc.gov/html/dot/downloads/pdf/2014-09-03-bicycle-path-data-analysis.pdf

standard, signed route, sharrows: http://bikingrules.org/biking/laneprimer/

In [9]:
ft_score = []
for row in stations_bike_quality['ft_facilit']:
    if row == 'Greenway':
        ft_score.append(5)
    elif row == 'Signed Route':
        ft_score.append(5)
    elif row == 'Ped Plaza':
        ft_score.append(5)
    elif row == 'Protected Path':
        ft_score.append(4)
    elif row == 'Bike-Friendly Parking':
        ft_score.append(4)
    elif row == 'Curbside':
        ft_score.append(3)
    elif row == 'Standard':
        ft_score.append(3)
    elif row == 'Sidewalk':
        ft_score.append(2)
    elif row == 'Opposite Sidewalk':
        ft_score.append(2)
    elif row == 'Sharrows':
        ft_score.append(1)
    else:
        ft_score.append(0)
stations_bike_quality['ft_facilit_score'] = ft_score

In [10]:
tf_score = []
for row in stations_bike_quality['tf_facilit']:
    if row == 'Greenway':
        tf_score.append(5)
    elif row == 'Signed Route':
        tf_score.append(5)
    elif row == 'Ped Plaza':
        tf_score.append(5)
    elif row == 'Protected Path':
        tf_score.append(4)
    elif row == 'Bike-Friendly Parking':
        tf_score.append(4)
    elif row == 'Curbside':
        tf_score.append(3)
    elif row == 'Standard':
        tf_score.append(3)
    elif row == 'Sharrows/Protected Path':
        tf_score.append(2.5)
    elif row == 'Sidewalk':
        tf_score.append(2)
    elif row == 'Opposite Sidewalk':
        tf_score.append(2)
    elif row == 'Sharrows/Standard':
        tf_score.append(1.5)
    elif row == 'Sharrows':
        tf_score.append(1)
    else:
        tf_score.append(0)
stations_bike_quality['tf_facilit_score'] = tf_score

In [11]:
stations_bike_quality.head()

,Station_id,Station_Name,Location,Latitude,Longitude,geometry,index_right,OBJECTID,SegmentID,allclasses,...,comments,fromstreet,ft_facilit,lanecount,onoffst,street,tf_facilit,tostreet,ft_facilit_score,tf_facilit_score
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9448,9490,0031767,II,...,None,BEACH ST,None,1.0,ON,W BROADWAY,Standard,READE ST,0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9451,9493,0031796,II,...,None,BEACH ST,None,1.0,ON,W BROADWAY,Standard,READE ST,0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9488,9530,0110911,II,...,None,BEACH ST,None,1.0,ON,VARICK ST,Standard,W BROADWAY,0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9450,9492,0031798,II,...,None,BEACH ST,None,1.0,ON,W BROADWAY,Standard,READE ST,0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9487,9529,0031789,II,...,None,BEACH ST,None,1.0,ON,VARICK ST,Standard,W BROADWAY,0,3.0


In [12]:
stations_bike_cut = stations_bike_quality.drop(['index_right', 'bikedir', 'OBJECTID', 'allclasses', 'comments', 'fromstreet', 'onoffst', 'street', 'tostreet'], axis=1)
#stations_bike_cut.head()
print(len(stations_bike_cut))

1049


In [14]:
nums = []
nums2 = []
for elem in stations_bike_cut['ft_facilit_score']:
    nums.append(elem)
#print(len(nums))

for elem in stations_bike_cut['tf_facilit_score']:
    nums2.append(elem)
#print(len(nums2))

nums_added = []
for i in range(0,len(stations_bike_cut)):
    if nums[i] > 0 and nums2[i] > 0:
        nums_added.append((nums[i] + nums2[i]) / 2)
    elif nums[i] > 0 and nums2[i] == 0:
        nums_added.append(nums[i])
    elif nums[i] == 0 and nums2[i] > 0:
        nums_added.append(nums2[i])

#print(len(nums_added))
stations_bike_cut['metric'] = nums_added
stations_bike_cut.head()

,Station_id,Station_Name,Location,Latitude,Longitude,geometry,SegmentID,boro,ft_facilit,lanecount,tf_facilit,ft_facilit_score,tf_facilit_score,metric
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",0031767,1.0,None,1.0,Standard,0,3.0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",0031796,1.0,None,1.0,Standard,0,3.0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",0110911,1.0,None,1.0,Standard,0,3.0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",0031798,1.0,None,1.0,Standard,0,3.0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",0031789,1.0,None,1.0,Standard,0,3.0,3.0


In [19]:
#groupby and take the average of the station

In [15]:
stations_bikes = pd.DataFrame(stations_bike_cut['metric'].groupby([stations_bike_cut['Station_id']]).mean())
stations_bikes = stations_bikes.reset_index()
stations_bikes.head()

,Station_id,metric
0,79,3.0
1,116,4.0
2,119,4.0
3,120,4.0
4,127,4.0


In [16]:
stations_bikes.to_csv('../data/processed/bike-lane-quality.csv')